# Mahalanobis outlier detector deployment

Wrap a Mahalanobis anomaly detection model for use as a prediction microservice in seldon-core and deploy on seldon-core running on minikube or a Kubernetes cluster using GCP.

## Dependencies

- [helm](https://github.com/helm/helm)
- [minikube](https://github.com/kubernetes/minikube)
- [s2i](https://github.com/openshift/source-to-image) >= 1.1.13

python packages:
- scikit-learn: pip install scikit-learn --> 0.20.1

## Task

The outlier detector needs to detect computer network intrusions using TCP dump data for a local-area network (LAN) simulating a typical U.S. Air Force LAN. A connection is a sequence of TCP packets starting and ending at some well defined times, between which data flows to and from a source IP address to a target IP address under some well defined protocol. Each connection is labeled as either normal, or as an attack. 

There are 4 types of attacks in the dataset:
- DOS: denial-of-service, e.g. syn flood;
- R2L: unauthorized access from a remote machine, e.g. guessing password;
- U2R:  unauthorized access to local superuser (root) privileges;
- probing: surveillance and other probing, e.g., port scanning.
    
The dataset contains about 5 million connection records.

There are 3 types of features:
- basic features of individual connections, e.g. duration of connection
- content features within a connection, e.g. number of failed log in attempts
- traffic features within a 2 second window, e.g. number of connections to the same host as the current connection

The outlier detector is only using the continuous (18 out of 41) features.

## Test using Kubernetes cluster on GCP or Minikube

Run the outlier detector as a model or a transformer. If you want to run the anomaly detector as a transformer, change the SERVICE_TYPE variable from MODEL to TRANSFORMER [here](./.s2i/environment), set MODEL = False and change ```OutlierMahalanobis.py``` to:

```python
from CoreMahalanobis import CoreMahalanobis

class OutlierMahalanobis(CoreMahalanobis):
    """ Outlier detection using the Mahalanobis distance.
    
    Parameters
    ----------
        threshold (float) : Mahalanobis distance threshold used to classify outliers
        n_components (int) : number of principal components used
        n_stdev (float) : stdev used for feature-wise clipping of observations
        start_clip (int) : number of observations before clipping is applied
        max_n (int) : algorithm behaves as if it has seen at most max_n points
    """
    def __init__(self,threshold=25,n_components=3,n_stdev=3,start_clip=50,max_n=-1):
        
        super().__init__(threshold=threshold,n_components=n_components,n_stdev=n_stdev,
                         start_clip=start_clip,max_n=max_n)
```

In [1]:
MODEL = True

Pick Kubernetes cluster on GCP or Minikube.

In [2]:
MINIKUBE = True

In [3]:
if MINIKUBE:
    !minikube start --memory 4096 --feature-gates=CustomResourceValidation=true \
    --extra-config=apiserver.Authorization.Mode=RBAC
else:
    !gcloud container clusters get-credentials standard-cluster-1 --zone europe-west1-b --project seldon-demos

Fetching cluster endpoint and auth data.
kubeconfig entry generated for standard-cluster-1.


Create a cluster-wide cluster-admin role assigned to a service account named “default” in the namespace “kube-system”.

In [4]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin \
--serviceaccount=kube-system:default

Error from server (AlreadyExists): clusterrolebindings.rbac.authorization.k8s.io "kube-system-cluster-admin" already exists


In [5]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


Add current context details to the configuration file in the seldon namespace.

In [6]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "gke_seldon-demos_europe-west1-b_standard-cluster-1" modified.


Create tiller service account and give it a cluster-wide cluster-admin role.

In [7]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

Error from server (AlreadyExists): serviceaccounts "tiller" already exists
Error from server (AlreadyExists): clusterrolebindings.rbac.authorization.k8s.io "tiller" already exists
$HELM_HOME has been configured at /home/arnaud/.helm.
(Use --client-only to suppress this message, or --upgrade to upgrade Tiller to the current version.)
Happy Helming!


Check deployment rollout status and deploy seldon/spartakus helm charts.

In [8]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

deployment "tiller-deploy" successfully rolled out


In [9]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd \
    --set usage_metrics.enabled=true

Error: a release named seldon-core-crd already exists.
Run: helm ls --all seldon-core-crd; to check the status of the release
Or run: helm del --purge seldon-core-crd; to delete it


In [10]:
!helm install ../../../helm-charts/seldon-core --name seldon-core \
        --namespace seldon \
        --set ambassador.enabled=true

Error: a release named seldon-core already exists.
Run: helm ls --all seldon-core; to check the status of the release
Or run: helm del --purge seldon-core; to delete it


Check deployment rollout status for seldon core.

In [11]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager -n seldon
!kubectl rollout status deploy/seldon-core-seldon-apiserver -n seldon

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out


If Minikube used: create docker image for outlier detector inside Minikube using s2i. Besides the transformer image and the demo specific model image, the general model image for the Mahalanobis outlier detector is also available from Docker Hub as ***seldonio/outlier-mahalanobis-model:0.1***.

In [12]:
if MINIKUBE & MODEL:
    !eval $(minikube docker-env) && \
    s2i build . seldonio/seldon-core-s2i-python3:0.4 seldonio/outlier-mahalanobis-model-demo:0.1
elif MINIKUBE:
    !eval $(minikube docker-env) && \
    s2i build . seldonio/seldon-core-s2i-python3:0.4 seldonio/outlier-mahalanobis-transformer:0.1

Install outlier detector helm charts either as a model or transformer and set *threshold*, *n_components*, *n_stdev* and *start_clip* hyperparameter values.

In [15]:
if MODEL:
    !helm install ../../../helm-charts/seldon-od-model \
        --name outlier-detector \
        --namespace=seldon \
        --set model.type=mahalanobis \
        --set model.mahalanobis.image.name=seldonio/outlier-mahalanobis-model-demo:0.1 \
        --set model.mahalanobis.threshold=25 \
        --set model.mahalanobis.n_components=3 \
        --set model.mahalanobis.n_stdev=3 \
        --set model.mahalanobis.start_clip=50 \
        --set oauth.key=oauth-key \
        --set oauth.secret=oauth-secret \
        --set replicas=1
else:
    !helm install ../../../helm-charts/seldon-od-transformer \
        --name outlier-detector \
        --namespace=seldon \
        --set outlierDetection.enabled=true \
        --set outlierDetection.name=outlier-mahalanobis \
        --set outlierDetection.type=mahalanobis \
        --set outlierDetection.mahalanobis.image.name=seldonio/outlier-mahalanobis-transformer:0.1 \
        --set outlierDetection.mahalanobis.threshold=25 \
        --set outlierDetection.mahalanobis.n_components=3 \
        --set outlierDetection.mahalanobis.n_stdev=3 \
        --set outlierDetection.mahalanobis.start_clip=50 \
        --set oauth.key=oauth-key \
        --set oauth.secret=oauth-secret \
        --set model.image.name=seldonio/mock_classifier:1.0

NAME:   outlier-detector
LAST DEPLOYED: Fri Feb  1 15:13:32 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME              AGE
outlier-detector  0s




## Port forward Ambassador

Run command in terminal:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

## Import rest requests, load data and test requests

In [16]:
from utils import get_payload, rest_request_ambassador, send_feedback_rest, get_kdd_data, generate_batch

data = get_kdd_data(percent10=True) # load dataset
print(data.shape)

(494021, 19)


Generate a random batch from the data

In [17]:
import numpy as np

samples = 1
fraction_outlier = 0.
X, labels = generate_batch(data,samples,fraction_outlier)
print(X.shape)
print(labels.shape)

(1, 18)
(1,)


Test the rest requests with the generated data. It is important that the order of requests is respected. First we make predictions, then we get the "true" labels back using the feedback request. If we do not respect the order and eg keep making predictions without getting the feedback for each prediction, there will be a mismatch between the predicted and "true" labels. This will result in errors in the produced metrics.

In [18]:
request = get_payload(X)

In [19]:
response = rest_request_ambassador("outlier-detector",request,endpoint="localhost:8003")

200
{
  "meta": {
    "puid": "d2126o8fno0tu7qct3er9sr1no",
    "tags": {
      "outlier-predictions": [0.0]
    },
    "routing": {
      "outlier-mahalanobis": -1
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0",
      "outlier-mahalanobis": "seldonio/outlier-mahalanobis-transformer:0.1"
    },
    "metrics": [{
      "key": "is_outlier",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "outlier_score",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "nb_outliers",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "fraction_outliers",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "observation",
      "type": "GAUGE",
      "value": 1.0,
      "tags": {
      }
    }, {
      "key": "threshold",
      "type": "GAUGE",
      "value": 25.0,
      "tags": {
      }
    }]
  },
  "data": {
    "name

If the outlier detector is used as a transformer, the output of the anomaly detection is added as part of the metadata. If it is used as a model, we send model feedback to retrieve custom performance metrics.

In [20]:
if MODEL:
    send_feedback_rest("outlier-detector",request,response,0,labels,endpoint="localhost:8003")

## Analytics

Install the helm charts for prometheus and the grafana dashboard

In [ ]:
!helm install ../../../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --namespace seldon

## Port forward Grafana dashboard

Run command in terminal:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n seldon 3000:3000
```

You can then view an analytics dashboard inside the cluster at http://localhost:3000/dashboard/db/prediction-analytics?refresh=5s&orgId=1. Your IP address may be different. get it via minikube ip. Login with:

Username : admin

password : password (as set when starting seldon-core-analytics above)

Import the outlier-detector-md dashboard from ../../../helm-charts/seldon-core-analytics/files/grafana/configs.

## Run simulation

- Sample random network intrusion data with a certain outlier probability.
- Get payload for the observation.
- Make a prediction.
- Send the "true" label with the feedback if the detector is run as a model.

It is important that the prediction-feedback order is maintained. Otherwise there will be a mismatch between the predicted and "true" labels.

View the progress on the grafana "Outlier Detection" dashboard. Most metrics need the outlier detector to be run as a model since they need model feedback.

In [21]:
import time
n_requests = 100
samples = 1
for i in range(n_requests):
    fraction_outlier = .1
    X, labels = generate_batch(data,samples,fraction_outlier)
    request = get_payload(X)
    response = rest_request_ambassador("outlier-detector",request,endpoint="localhost:8003")
    if MODEL:
        send_feedback_rest("outlier-detector",request,response,0,labels,endpoint="localhost:8003")
    time.sleep(1)

200
{
  "meta": {
    "puid": "2te4e071gm6ve055c8kdedqibn",
    "tags": {
      "outlier-predictions": [0.0]
    },
    "routing": {
      "outlier-mahalanobis": -1
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0",
      "outlier-mahalanobis": "seldonio/outlier-mahalanobis-transformer:0.1"
    },
    "metrics": [{
      "key": "is_outlier",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "outlier_score",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "nb_outliers",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "fraction_outliers",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "observation",
      "type": "GAUGE",
      "value": 2.0,
      "tags": {
      }
    }, {
      "key": "threshold",
      "type": "GAUGE",
      "value": 25.0,
      "tags": {
      }
    }]
  },
  "data": {
    "name

200
{
  "meta": {
    "puid": "binst1ke42eo9eqpuedp8co2tv",
    "tags": {
      "outlier-predictions": [0.0]
    },
    "routing": {
      "outlier-mahalanobis": -1
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0",
      "outlier-mahalanobis": "seldonio/outlier-mahalanobis-transformer:0.1"
    },
    "metrics": [{
      "key": "is_outlier",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "outlier_score",
      "type": "GAUGE",
      "value": 4.991297,
      "tags": {
      }
    }, {
      "key": "nb_outliers",
      "type": "GAUGE",
      "value": 2.0,
      "tags": {
      }
    }, {
      "key": "fraction_outliers",
      "type": "GAUGE",
      "value": 0.2,
      "tags": {
      }
    }, {
      "key": "observation",
      "type": "GAUGE",
      "value": 10.0,
      "tags": {
      }
    }, {
      "key": "threshold",
      "type": "GAUGE",
      "value": 25.0,
      "tags": {
      }
    }]
  },
  "data": {
   

200
{
  "meta": {
    "puid": "jr18tqde6fd8kmn0a9m14ebd82",
    "tags": {
      "outlier-predictions": [0.0]
    },
    "routing": {
      "outlier-mahalanobis": -1
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0",
      "outlier-mahalanobis": "seldonio/outlier-mahalanobis-transformer:0.1"
    },
    "metrics": [{
      "key": "is_outlier",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "outlier_score",
      "type": "GAUGE",
      "value": 2.315602,
      "tags": {
      }
    }, {
      "key": "nb_outliers",
      "type": "GAUGE",
      "value": 3.0,
      "tags": {
      }
    }, {
      "key": "fraction_outliers",
      "type": "GAUGE",
      "value": 0.16666667,
      "tags": {
      }
    }, {
      "key": "observation",
      "type": "GAUGE",
      "value": 18.0,
      "tags": {
      }
    }, {
      "key": "threshold",
      "type": "GAUGE",
      "value": 25.0,
      "tags": {
      }
    }]
  },
  "data"

200
{
  "meta": {
    "puid": "hmechuihqh3k7d8df0vgfnj5ab",
    "tags": {
      "outlier-predictions": [0.0]
    },
    "routing": {
      "outlier-mahalanobis": -1
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0",
      "outlier-mahalanobis": "seldonio/outlier-mahalanobis-transformer:0.1"
    },
    "metrics": [{
      "key": "is_outlier",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "outlier_score",
      "type": "GAUGE",
      "value": 5.1056695,
      "tags": {
      }
    }, {
      "key": "nb_outliers",
      "type": "GAUGE",
      "value": 4.0,
      "tags": {
      }
    }, {
      "key": "fraction_outliers",
      "type": "GAUGE",
      "value": 0.15384616,
      "tags": {
      }
    }, {
      "key": "observation",
      "type": "GAUGE",
      "value": 26.0,
      "tags": {
      }
    }, {
      "key": "threshold",
      "type": "GAUGE",
      "value": 25.0,
      "tags": {
      }
    }]
  },
  "data

200
{
  "meta": {
    "puid": "cdl2rt4dj9j4fgkuekgfoviklo",
    "tags": {
      "outlier-predictions": [0.0]
    },
    "routing": {
      "outlier-mahalanobis": -1
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0",
      "outlier-mahalanobis": "seldonio/outlier-mahalanobis-transformer:0.1"
    },
    "metrics": [{
      "key": "is_outlier",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "outlier_score",
      "type": "GAUGE",
      "value": 10.892465,
      "tags": {
      }
    }, {
      "key": "nb_outliers",
      "type": "GAUGE",
      "value": 4.0,
      "tags": {
      }
    }, {
      "key": "fraction_outliers",
      "type": "GAUGE",
      "value": 0.11764706,
      "tags": {
      }
    }, {
      "key": "observation",
      "type": "GAUGE",
      "value": 34.0,
      "tags": {
      }
    }, {
      "key": "threshold",
      "type": "GAUGE",
      "value": 25.0,
      "tags": {
      }
    }]
  },
  "data

...

200
{
  "meta": {
    "puid": "gabrfh4f18r6tl14eunmbht2ff",
    "tags": {
      "outlier-predictions": [0.0]
    },
    "routing": {
      "outlier-mahalanobis": -1
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0",
      "outlier-mahalanobis": "seldonio/outlier-mahalanobis-transformer:0.1"
    },
    "metrics": [{
      "key": "is_outlier",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "outlier_score",
      "type": "GAUGE",
      "value": 8.393401,
      "tags": {
      }
    }, {
      "key": "nb_outliers",
      "type": "GAUGE",
      "value": 5.0,
      "tags": {
      }
    }, {
      "key": "fraction_outliers",
      "type": "GAUGE",
      "value": 0.07575758,
      "tags": {
      }
    }, {
      "key": "observation",
      "type": "GAUGE",
      "value": 66.0,
      "tags": {
      }
    }, {
      "key": "threshold",
      "type": "GAUGE",
      "value": 25.0,
      "tags": {
      }
    }]
  },
  "data"

200
{
  "meta": {
    "puid": "jnnqut0qhlcuvliu622rmvi8k4",
    "tags": {
      "outlier-predictions": [1.0]
    },
    "routing": {
      "outlier-mahalanobis": -1
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0",
      "outlier-mahalanobis": "seldonio/outlier-mahalanobis-transformer:0.1"
    },
    "metrics": [{
      "key": "is_outlier",
      "type": "GAUGE",
      "value": 1.0,
      "tags": {
      }
    }, {
      "key": "outlier_score",
      "type": "GAUGE",
      "value": 149.97888,
      "tags": {
      }
    }, {
      "key": "nb_outliers",
      "type": "GAUGE",
      "value": 7.0,
      "tags": {
      }
    }, {
      "key": "fraction_outliers",
      "type": "GAUGE",
      "value": 0.0945946,
      "tags": {
      }
    }, {
      "key": "observation",
      "type": "GAUGE",
      "value": 74.0,
      "tags": {
      }
    }, {
      "key": "threshold",
      "type": "GAUGE",
      "value": 25.0,
      "tags": {
      }
    }]
  },
  "data"

200
{
  "meta": {
    "puid": "cqpb87mq72pfr182hr25d5emgs",
    "tags": {
      "outlier-predictions": [1.0]
    },
    "routing": {
      "outlier-mahalanobis": -1
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0",
      "outlier-mahalanobis": "seldonio/outlier-mahalanobis-transformer:0.1"
    },
    "metrics": [{
      "key": "is_outlier",
      "type": "GAUGE",
      "value": 1.0,
      "tags": {
      }
    }, {
      "key": "outlier_score",
      "type": "GAUGE",
      "value": 161.58379,
      "tags": {
      }
    }, {
      "key": "nb_outliers",
      "type": "GAUGE",
      "value": 8.0,
      "tags": {
      }
    }, {
      "key": "fraction_outliers",
      "type": "GAUGE",
      "value": 0.09756097,
      "tags": {
      }
    }, {
      "key": "observation",
      "type": "GAUGE",
      "value": 82.0,
      "tags": {
      }
    }, {
      "key": "threshold",
      "type": "GAUGE",
      "value": 25.0,
      "tags": {
      }
    }]
  },
  "data

200
{
  "meta": {
    "puid": "b9idsnma21bjrnuq0k2dm720jl",
    "tags": {
      "outlier-predictions": [0.0]
    },
    "routing": {
      "outlier-mahalanobis": -1
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0",
      "outlier-mahalanobis": "seldonio/outlier-mahalanobis-transformer:0.1"
    },
    "metrics": [{
      "key": "is_outlier",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "outlier_score",
      "type": "GAUGE",
      "value": 3.2748032,
      "tags": {
      }
    }, {
      "key": "nb_outliers",
      "type": "GAUGE",
      "value": 9.0,
      "tags": {
      }
    }, {
      "key": "fraction_outliers",
      "type": "GAUGE",
      "value": 0.1,
      "tags": {
      }
    }, {
      "key": "observation",
      "type": "GAUGE",
      "value": 90.0,
      "tags": {
      }
    }, {
      "key": "threshold",
      "type": "GAUGE",
      "value": 25.0,
      "tags": {
      }
    }]
  },
  "data": {
  

200
{
  "meta": {
    "puid": "kqj54skbn3gr0jiloh2k8os6bu",
    "tags": {
      "outlier-predictions": [0.0]
    },
    "routing": {
      "outlier-mahalanobis": -1
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0",
      "outlier-mahalanobis": "seldonio/outlier-mahalanobis-transformer:0.1"
    },
    "metrics": [{
      "key": "is_outlier",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "outlier_score",
      "type": "GAUGE",
      "value": 1.7463398,
      "tags": {
      }
    }, {
      "key": "nb_outliers",
      "type": "GAUGE",
      "value": 11.0,
      "tags": {
      }
    }, {
      "key": "fraction_outliers",
      "type": "GAUGE",
      "value": 0.1122449,
      "tags": {
      }
    }, {
      "key": "observation",
      "type": "GAUGE",
      "value": 98.0,
      "tags": {
      }
    }, {
      "key": "threshold",
      "type": "GAUGE",
      "value": 25.0,
      "tags": {
      }
    }]
  },
  "data

In [ ]:
if MINIKUBE:
    !minikube delete